In [28]:
import json
import torch

from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.constraints import GreaterThan
from gpytorch.distributions import MultivariateNormal
from gpytorch.priors import GammaPrior, NormalPrior
from gpytorch.likelihoods import GaussianLikelihood, FixedNoiseGaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

from utils.bo_utils import GPModel, plot_optimization
from bayesian_optimization import BayesianOptimization

ImportError: cannot import name 'expected_improvement' from 'utils.bo_utils' (/home/toelle/Dropbox/Projects/midl/utils/bo_utils.py)

In [25]:
log_dir = "../bo_exps/bo"
results = torch.load(f"{log_dir}/results.pt")
params_space = results["p_space"]

In [26]:
config = "./configs/bo_prior_sigma"
with open(config + ".json") as f:
    config = json.load(f)
    
lengthscale_prior = config["lengthscale_prior"] if "lengthscale_prior" in list(config.keys()) else dict(concentration=0.3, rate=1.)
lengthscale_constraint = config["lengthscale_constraint"] if "lengthscale_constraint" in list(config.keys()) else 0.05
mean_prior = config["mean_prior"] if "mean_prior" in list(config.keys()) else dict(loc=25., scale=2.)
noise_prior = config["noise_prior"] if "noise_prior" in list(config.keys()) else dict(concentration=1e-2, rate=100.)

params = {p["name"]: p["bounds"] for p in config["parameter"]}
bounds = np.array(list(params.values()))

NameError: name 'np' is not defined

In [23]:
import pdb;pdb.set_trace()
likelihood = GaussianLikelihood(noise_prior=GammaPrior(**noise_prior))

for i in range(len(results) - 1):
    vals = results[i]
    
    params_samples = vals["p_samples"]
    cost_samples = vals["c_samples"]
    
    model = GPModel(
        params_samples, cost_samples, likelihood, lengthscale_prior,
        lengthscale_constraint, mean_prior
    ).double()
    
    model.load_state_dict(vals["state_dict"])
    
    plot_optimization(
        model, likelihood, expected_improvement,
        next_params, params_space,
        params_samples, cost_samples, path
    )